In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from numpy import logical_and as npand
from numpy import logical_or as npor
import matplotlib

In [ ]:
from sympy import (symbols, pi, I, E, cos, sin, exp, tan, simplify, expand, factor, collect,
                   apart, cancel, expand_trig, diff, Derivative, Function, integrate, limit,
                   series, Eq, solve, dsolve, Matrix, N, preorder_traversal, Float, solve_linear_system,
                   eye, zeros, lambdify, Symbol,hessian, sqrt, evalf)
from sympy.physics.mechanics import dynamicsymbols, init_vprinting

In [ ]:
init_vprinting()

In [ ]:
from sympy.physics.mechanics import ReferenceFrame, Point, LagrangesMethod, Lagrangian, inertia, RigidBody, dot

In [ ]:
N = ReferenceFrame('N')
origen = Point('origen')
origen.set_vel(N, 0)


In [ ]:
t = symbols('t')
t

In [ ]:
x, y, psi = dynamicsymbols('x y psi')
x, y, psi

In [ ]:
fi0, fi1, fi2, fi3 = dynamicsymbols('phi:4')
fi0, fi1, fi2, fi3

In [ ]:
CM = origen.locatenew('CM', x*N.x + y*N.y)
CM.set_vel(N, CM.pos_from(origen).dt(N))

In [ ]:
B = N.orientnew('B', 'Axis', (psi, N.z))

In [ ]:
m_r, m_w, i_r, i_w, l0, l1, r = symbols('m_r m_w i_r i_w l0 l1 r')

In [ ]:
I_r = inertia(B, 0, 0, i_r)
body = RigidBody('Body', CM, B, m_r, (I_r,CM))

In [ ]:
wheel_params = [
    [45,  l0,  l1, fi0],
    [-45, l0, -l1, fi1],
    [-45, -l0, l1, fi2],
    [45, -l0, -l1, fi3]
]

In [ ]:
wheel_axes = []
wheel_centers = []
I_w = inertia(B, 0, i_w, 0)
wheel_bodies = []
for ii in range(4):
    wheel_axis = B.orientnew(f'W{ii}', 'Axis', (wheel_params[ii][3], B.y))
    wheel_axes.append(wheel_axis)
    wheel_center = CM.locatenew(f'CMW{ii}', wheel_params[ii][1]*B.x + wheel_params[ii][2]*B.y)
    wheel_center.set_vel(N, wheel_center.v2pt_theory(CM, N, B))
    wheel_centers.append(wheel_center)
    wheel_body = RigidBody(f'Wheel{ii}', wheel_center, wheel_axis, 0, (I_w,wheel_center))
    wheel_bodies.append(wheel_body)
wheel_axes, wheel_centers, wheel_bodies

In [ ]:
lag_f = simplify(Lagrangian(N, body, *wheel_bodies))
lag_f

## Restrictions:

In [ ]:
from optibot.symbolic import integerize

In [ ]:
non_hol_rest = []
for ii in range(4):
    angle = np.radians(wheel_params[ii][0])
    aux_syst = B.orientnew('aux', 'Axis', (angle, B.z))
    contact_p = wheel_centers[ii].locatenew('contact_p', -r*N.z)
    contact_p.set_vel(N, contact_p.v2pt_theory(wheel_centers[ii], N, wheel_axes[ii]))
    raw_restr = dot(contact_p.vel(N),aux_syst.y)
    non_hol_rest.append(simplify(integerize((2/2**0.5)* raw_restr)))

In [ ]:
Matrix(non_hol_rest)

## Lagrange system

In [ ]:
FL = []
for ii in range(4):
    u = symbols(f'u{ii}')
    FL.append((wheel_axes[ii], u*B.y))

In [ ]:
from optibot.symbolic import SimpLagrangesMethod

In [ ]:
LM_simp = SimpLagrangesMethod(lag_f, [x, y, psi, fi0, fi1, fi2, fi3] ,
                              forcelist=FL, frame=N, nonhol_coneqs=non_hol_rest)

In [ ]:
LM_simp.Q

In [ ]:
LM_simp.M

In [ ]:
LM_simp.phi_q

In [ ]:
LM_simp.H

In [ ]:
LM_simp.K

In [ ]:
LM_simp.Fa

In [ ]:
LM_simp.RHS_reduced

In [ ]:
LM_simp.calculate_RHS()

In [ ]:
LM_simp.calculate_RHS_full()

### Reference: based on Lagranges Method

In [ ]:
LM = LagrangesMethod(lag_f, [x, y, psi, fi0, fi1, fi2, fi3] , forcelist=FL, frame=N, nonhol_coneqs=non_hol_rest)

In [ ]:
LM.form_lagranges_equations()

### Reference: also works without constraints

In [ ]:
LM_simp_no_const = SimpLagrangesMethod(lag_f, [x, y, psi, fi0, fi1, fi2, fi3] ,
                              forcelist=FL, frame=N, nonhol_coneqs=None)

In [ ]:
LM_simp_no_const.calculate_RHS_full()

## Transformation into numeric functions

In [ ]:
from optibot.symbolic import print_funcs_RHS

In [ ]:
print_funcs_RHS(LM_simp.RHS_reduced[3:],[x, y, psi])

In [ ]:
from optibot.numpy import unpack
def F(x, u, params):
    x, y, psi, x_dot, y_dot, psi_dot = unpack(x)
    u_0, u_1, u_2, u_3 = unpack(u)
    i_r, i_w, l0, l1, m_r, r = params
    result = [x_dot, y_dot, psi_dot,]
    result.append((-4*i_w*psi_dot*y_dot + sqrt(2)*r*u_0*sin(psi + pi/4) + sqrt(2)*r*u_1*cos(psi + pi/4) + sqrt(2)*r*u_2*cos(psi + pi/4) + sqrt(2)*r*u_3*sin(psi + pi/4))/(4*i_w + m_r*r**2))
    result.append((4*i_w*psi_dot*x_dot - sqrt(2)*r*u_0*cos(psi + pi/4) + sqrt(2)*r*u_1*sin(psi + pi/4) + sqrt(2)*r*u_2*sin(psi + pi/4) - sqrt(2)*r*u_3*cos(psi + pi/4))/(4*i_w + m_r*r**2))
    result.append(r*(-l0*u_0 + l0*u_1 - l0*u_2 + l0*u_3 - l1*u_0 + l1*u_1 - l1*u_2 + l1*u_3)/(i_r*r**2 + 4*i_w*l0**2 + 8*i_w*l0*l1 + 4*i_w*l1**2))

    return result

In [ ]:
R = []
for exp in F([1,1,1,1,1,1],[1,1,1,1],[1,1,1,1,1,1]):
    try:
        R.append(exp.evalf())
    except:
        R.append(exp)
R

In [ ]:
from optibot.numpy import RHS2numpy

In [ ]:
np_func = RHS2numpy(LM_simp.RHS_reduced[3:],[x, y, psi])

In [ ]:
np_func([1,1,1,1,1,1],[1,1,1,1],[1,1,1,1,1,1])

## Integración con Casadi

In [ ]:
import casadi as cas

In [ ]:
from optibot.casadi import RHS2casF

In [ ]:
from optibot.schemes import euler_restr, trapz_restr, trapz_mod_restr, hs_mod_restr, hs_restr
from optibot.casadi import restriction2casadi

In [ ]:
F_cas_simp = RHS2casF(LM_simp.RHS_reduced[3:],[x, y, psi])

In [ ]:
F_cas_simp([1,1,1,1,1,1],[1,1,1,1],[1,1,1,1,1,1])

In [ ]:
N = 200

In [ ]:
opti = cas.Opti()
opti.solver('ipopt')

In [ ]:
X = opti.variable(N+1,6)
U = opti.variable(N+1,4)
T = opti.parameter()
u_m = opti.parameter(4)
Params = opti.parameter(6)

In [ ]:
cost = cas.sum2(cas.sum1(U**2))
#cost = cas.sum1(cas.cos(X[:,1])) + cas.sum1(X[:,0]**2)
#cost = cas.sum1(cas.cos(X[:,1]))
opti.minimize(cost)

In [ ]:
opti.subject_to(X[0,:].T == [0, 0, 0, 0, 0, 0])
opti.subject_to(X[-1,:].T == [2, 1.5, 1, 0, 0, 0])
opti.subject_to(U[0,:].T == [0, 0, 0, 0])
opti.subject_to(U[-1,:].T == [0, 0, 0, 0])

In [ ]:
#restriction = restriction2casadi(euler_restr, F_cas, 3, 4, 6)
restriction = restriction2casadi(trapz_restr, F_cas_simp, 3, 4, 6)
#restriction = restriction2casadi(hs_restr, F_cas, 3, 4, 6)

In [ ]:
for ii in range(N):
    opti.subject_to(restriction(X[ii,:], X[ii+1,:], U[ii,:], U[ii+1,:],T/N, Params)==0)
    opti.subject_to(opti.bounded(-u_m[0],U[ii, 0],u_m[0]))
    opti.subject_to(opti.bounded(-u_m[1],U[ii, 1],u_m[1]))
    opti.subject_to(opti.bounded(-u_m[2],U[ii, 2],u_m[2]))
    opti.subject_to(opti.bounded(-u_m[3],U[ii, 3],u_m[3]))

In [ ]:
opti.set_value(T, 12)
max_par = 2.0
opti.set_value(u_m, [max_par for ii in range(4)])

m_r = 15.75
l0 = 0.2096
l1 = 0.2096
i_r = 0.461
i_w = 0.00266
r = 0.0667

opti.set_value(Params, [i_r, i_w, l0, l1, m_r, r])

In [ ]:
sol = opti.solve()

In [ ]:
xx_simp = sol.value(X)
uu_simp = sol.value(U) 

In [ ]:
plt.figure(figsize=[15,10])
for ii in range(3):
    plt.plot(xx_simp[:,ii])

In [ ]:
plt.figure(figsize=[15,10])
for ii in range(4):
    plt.plot(uu_simp[:,ii])

In [ ]:
from optibot.schemes import integrate_euler, integrate_hs, integrate_trapz, integrate_rk4

In [ ]:
compr_sol_simp = integrate_trapz(
    xx_simp[0,:],
    uu_simp,
    np_func,
    sol.value(T)/N,
    [i_r, i_w, l0, l1, m_r, r]
)

In [ ]:
plt.figure(figsize=[15,10])
for ii in range(3):
    plt.plot(compr_sol_simp[:,ii])